In [37]:
import pandas as pd
import os

In [38]:
# Import Python Toolbox and check which version you are using
import acestbx_pyace
print("This notebook uses Toolbox_Python_{}".format(acestbx_pyace.__version__))

This notebook uses Toolbox_Python_20211014


In [39]:
# Import the Toolbox functionalities which will be used in this notebook
from acestbx_pyace.datachecks import DataAudit
from acestbx_pyace.modelling import SelectionSet

Import Data     

In [40]:
projpath = r"C:\Users\User\Desktop\Part 1 - Bank Case Study Demo - Days 1 to 6\Part 2 - Independent Case Study - Days 7 to 10"

In [41]:
# Define the full filepath 
df = os.path.join(projpath, r"Retail_data.csv")
print(df)

C:\Users\User\Desktop\Part 1 - Bank Case Study Demo - Days 1 to 6\Part 2 - Independent Case Study - Days 7 to 10\Retail_data.csv


In [42]:
df = pd.read_csv(df, keep_default_na=False, encoding = "utf8")

In [43]:
df.head()

,Unique_ID,Extract_Date,Age_Of_Account,Num_Periods_Since_1st_Statement,Type_Of_Application,Days_Since_Most_Recent_Order,Num_Orders_L4_6P,Ave_Item_Value_L13P,Num_Items_Ordered_L13P,Ave_Num_Items_Per_Order_L4_6P,...,Num_2_Or_More_Missed_Pmnts_L4_6P,Value_HR_Orders_Perc_Orders_L13P,Num_Account_Reviews_L13P,Ave_Order_Value_L13P,Num_Active_Accs_Not_In_Arrears,Bureau_Index_Retail,WS_L6M_Accs_Open_More_Than_12M,Num_Inactive_Accs_Sole_Joint,Good_Bad_Flag,Random_Number
0,1,20090917,122,128,U,385,0,0,0,0,...,1,0,13,0,9,70,2,9,G,0.739020
1,2,20100318,124,134,U,187,0,18,10,0,...,3,0,12,36,2,40,3,7,G,0.272479
2,3,20090917,122,128,U,257,0,33,7,0,...,1,0,9,77,0,40,N,5,B,0.709533
3,4,20091203,134,130,U,999999,0,0,0,0,...,3,0,13,0,1,40,0,6,G,0.319164
4,5,20100318,243,134,U,124,2,27,24,9,...,1,13,13,159,3,40,2,5,G,0.367853


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7608 entries, 0 to 7607
Data columns (total 35 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unique_ID                         7608 non-null   int64  
 1   Extract_Date                      7608 non-null   int64  
 2   Age_Of_Account                    7608 non-null   int64  
 3   Num_Periods_Since_1st_Statement   7608 non-null   int64  
 4   Type_Of_Application               7608 non-null   object 
 5   Days_Since_Most_Recent_Order      7608 non-null   int64  
 6   Num_Orders_L4_6P                  7608 non-null   int64  
 7   Ave_Item_Value_L13P               7608 non-null   int64  
 8   Num_Items_Ordered_L13P            7608 non-null   int64  
 9   Ave_Num_Items_Per_Order_L4_6P     7608 non-null   int64  
 10  Value_Last_Payment                7608 non-null   int64  
 11  Num_Payments_CTD                  7608 non-null   int64  
 12  Days_S

In [45]:
df.dtypes

Unique_ID                             int64
Extract_Date                          int64
Age_Of_Account                        int64
Num_Periods_Since_1st_Statement       int64
Type_Of_Application                  object
Days_Since_Most_Recent_Order          int64
Num_Orders_L4_6P                      int64
Ave_Item_Value_L13P                   int64
Num_Items_Ordered_L13P                int64
Ave_Num_Items_Per_Order_L4_6P         int64
Value_Last_Payment                    int64
Num_Payments_CTD                      int64
Days_Since_Most_Recent_Payment        int64
Num_Payments_To_Date                  int64
Num_Payments_L4_6P                    int64
Value_Payments_L3P                    int64
Value_Payments_L4_6P                  int64
Ave_Payment_Value_L4_6P               int64
Ave_Payment_Value_L13P                int64
Ave_Pmnt_Over_Bal_L13P_Perc           int64
Ave_Current_Balance_L4_6P             int64
Current_Bal_Over_Ave_Bal_L4_6P        int64
Curr_Bal_Over_Highest_Bal_L4_6P 

Duplicate Check

In [46]:
# Check for full duplicates
df['Duplicate_flag_all']=df.duplicated(list(df.columns))
df['Duplicate_flag_all'].value_counts()
# No full dups

False    7608
Name: Duplicate_flag_all, dtype: int64

In [47]:
# Check duplicates by Operation_id
df['Duplicate_Unique_ID']=df.duplicated('Unique_ID')
df['Duplicate_Unique_ID'].value_counts()
# No dups by ID

False    7608
Name: Duplicate_Unique_ID, dtype: int64

In [48]:
# Drop the 2 columns created for duplicate checks from the dataset
df = df.drop(['Duplicate_flag_all','Duplicate_Unique_ID'], axis=1).copy()
print(df.shape)

(7608, 35)


Apply Data Dictionary

In [49]:
def data_dictionary(df):
    df['Type_Of_Application'] = df['Type_Of_Application'].map(
            {'I': 'Internet Application',
             'T': 'Telephone Application',
             'C': 'Cross Sell Application',
             'U': 'Unknown Application'})
 
 
    df['WS_L6M_Accs_Open_More_Than_12M'] = df['WS_L6M_Accs_Open_More_Than_12M'].map(
            {'0': 'account not in arrears',
             'U': 'account not in arrears',
             '1': '1 month missed',
             '2': '2 months missed',
             '3': '3 months missed',
             '4': '4 months missed',
             '5': '5 months missed',
             '6': '6 months missed',
             '7': '7 months missed',
             '8': 'Defaulted Account with more than 6+ months missed',
             'T': 'No bureau information found at all',
             'N': 'No relevant bureau information found'})

In [50]:
data_dictionary(df)
df.head()

,Unique_ID,Extract_Date,Age_Of_Account,Num_Periods_Since_1st_Statement,Type_Of_Application,Days_Since_Most_Recent_Order,Num_Orders_L4_6P,Ave_Item_Value_L13P,Num_Items_Ordered_L13P,Ave_Num_Items_Per_Order_L4_6P,...,Num_2_Or_More_Missed_Pmnts_L4_6P,Value_HR_Orders_Perc_Orders_L13P,Num_Account_Reviews_L13P,Ave_Order_Value_L13P,Num_Active_Accs_Not_In_Arrears,Bureau_Index_Retail,WS_L6M_Accs_Open_More_Than_12M,Num_Inactive_Accs_Sole_Joint,Good_Bad_Flag,Random_Number
0,1,20090917,122,128,Unknown Application,385,0,0,0,0,...,1,0,13,0,9,70,2 months missed,9,G,0.739020
1,2,20100318,124,134,Unknown Application,187,0,18,10,0,...,3,0,12,36,2,40,3 months missed,7,G,0.272479
2,3,20090917,122,128,Unknown Application,257,0,33,7,0,...,1,0,9,77,0,40,No relevant bureau information found,5,B,0.709533
3,4,20091203,134,130,Unknown Application,999999,0,0,0,0,...,3,0,13,0,1,40,account not in arrears,6,G,0.319164
4,5,20100318,243,134,Unknown Application,124,2,27,24,9,...,1,13,13,159,3,40,2 months missed,5,G,0.367853


Perform Data Audit on the data and exclude any variables with %Missing > 90% and with 1 Unique Values.
o Explore the Data Audit report and decide which variables are not going to be used further for modelling

In [51]:
pd.read_excel(r"C:\Users\User\Desktop\Part 1 - Bank Case Study Demo - Days 1 to 6\Part 2 - Independent Case Study - Days 7 to 10\Data Audit v1.0.xlsx")

,#,Field Name,Classes,Missing
0,1,Age_Of_Account,233,0
1,2,Ave_Current_Balance_L4_6P,1268,0
2,3,Ave_Item_Value_L13P,188,0
3,4,Ave_Num_Items_Per_Order_L4_6P,30,0
4,5,Ave_Order_Value_L13P,419,0
5,6,Ave_Payment_Value_L13P,187,0
6,7,Ave_Payment_Value_L4_6P,189,0
7,8,Ave_Pmnt_Over_Bal_L13P_Perc,180,0
8,9,Bureau_Index_Retail,21,0
9,10,Current_Arrears_Amount,426,0


In [53]:
# initialize selection set
ss = SelectionSet()
ss.fit(df)

Creating selection set                            : 100%|██████████| 35/35 [00:00<00:00, 16185.30it/s]


In [55]:
# Remove some fields which we do not need in the Data Audit - such as the ID.
ss.remove(columns=['Unique_ID'])

In [56]:
ss.ss()

,name,type,classing_object,fc_type,fc_options,cc_type,cc_options,special_values,woe,continuous,metadata
Extract_Date,Extract_Date,integer,None,percent,"{'percent': 5, 'precision': None, 'rounding': ...",numeric,[],{},0,0,{}
Age_Of_Account,Age_Of_Account,integer,None,percent,"{'percent': 5, 'precision': None, 'rounding': ...",numeric,[],{},0,0,{}
Num_Periods_Since_1st_Statement,Num_Periods_Since_1st_Statement,integer,None,percent,"{'percent': 5, 'precision': None, 'rounding': ...",numeric,[],{},0,0,{}
Type_Of_Application,Type_Of_Application,string,None,single_values_char,"{'max_uniques': 200, 'order': 'alphabetical'}",character,[],{},0,0,{}
Days_Since_Most_Recent_Order,Days_Since_Most_Recent_Order,integer,None,percent,"{'percent': 5, 'precision': None, 'rounding': ...",numeric,[],{},0,0,{}
Num_Orders_L4_6P,Num_Orders_L4_6P,integer,None,percent,"{'percent': 5, 'precision': None, 'rounding': ...",numeric,[],{},0,0,{}
Ave_Item_Value_L13P,Ave_Item_Value_L13P,integer,None,percent,"{'percent': 5, 'precision': None, 'rounding': ...",numeric,[],{},0,0,{}
Num_Items_Ordered_L13P,Num_Items_Ordered_L13P,integer,None,percent,"{'percent': 5, 'precision': None, 'rounding': ...",numeric,[],{},0,0,{}
Ave_Num_Items_Per_Order_L4_6P,Ave_Num_Items_Per_Order_L4_6P,integer,None,percent,"{'percent': 5, 'precision': None, 'rounding': ...",numeric,[],{},0,0,{}
Value_Last_Payment,Value_Last_Payment,integer,None,percent,"{'percent': 5, 'precision': None, 'rounding': ...",numeric,[],{},0,0,{}


In [57]:
# Initialize data audit
data_audit = DataAudit(separate_zeros=True,
                       percent_threshold=5,
                       calculate_uniques=True,
                       max_uniques=100)

In [59]:
# Fit the data audit
data_audit.fit(ss,
               df)

Validating and fitting data...


Creating classing objects                         :   0%|          | 0/34 [00:00<?, ?it/s]<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:1: SyntaxWarning: "is" with a 

In [61]:
# Export the data audit as .xlsx file
path_da = os.path.join(projpath, r"04 Analysis\01 Initial Data Investigation\01 Data Audit")
data_audit.export(path_da,
                  name='Data Audit')

In [64]:
# View the summary output for each variable using the view_summary method and create a dataframe from the summary output
da_summary = data_audit.view_summary(display_only=False)
print(da_summary)

                            Variable     Type Label  Unique Values  Missings  \
0                       Extract_Date  integer     .            3.0       0.0   
1                     Age_Of_Account  integer     .          233.0       0.0   
2    Num_Periods_Since_1st_Statement  integer     .           79.0       0.0   
3                Type_Of_Application   string     .            4.0       0.0   
4       Days_Since_Most_Recent_Order  integer     .          750.0       0.0   
5                   Num_Orders_L4_6P  integer     .           17.0       0.0   
6                Ave_Item_Value_L13P  integer     .          188.0       0.0   
7             Num_Items_Ordered_L13P  integer     .          103.0       0.0   
8      Ave_Num_Items_Per_Order_L4_6P  integer     .           30.0       0.0   
9                 Value_Last_Payment  integer     .          197.0       0.0   
10                  Num_Payments_CTD  integer     .            8.0       0.0   
11    Days_Since_Most_Recent_Payment  in

Split data 

In [65]:
import sklearn                # Python's machine learning library
from sklearn.model_selection import train_test_split

In [67]:
# Set train_split_size to the % of the sample to be used in model development - expects decimal in (0,1]
train_split_size = 0.8
seed = 42

if train_split_size > 0 and train_split_size < 1:
    df_train, df_test = train_test_split(
        df.index,
        test_size=1 - train_split_size,
        random_state=seed,
        stratify=df['Good_Bad_Flag'])
    # Prepare the train and test samples
    df_train = df.loc[df_train, :]
    df_test = df.loc[df_test, :]
else:
    raise ValueError(f"train_split_size must be a fraction > 0 and <= 1")

In [68]:
# Check the shape of the created samples and the initial sample
print(df_train.shape)
print(df_test.shape)
print(df.shape)

(6086, 35)
(1522, 35)
(7608, 35)


In [71]:
# Check the number of goods and bads in the samples
print(df_train['Good_Bad_Flag'].value_counts())
print(df_test['Good_Bad_Flag'].value_counts())
print(df['Good_Bad_Flag'].value_counts())

B    3071
G    1575
I    1286
E     154
Name: Good_Bad_Flag, dtype: int64
B    768
G    394
I    322
E     38
Name: Good_Bad_Flag, dtype: int64
B    3839
G    1969
I    1608
E     192
Name: Good_Bad_Flag, dtype: int64


In [72]:
# Check if the percentage of bads is the same across all samples
print(3071/6086) # B/all samples from train
print(768/1522) # B/test
print(3839/7608) # B/ df.shape


0.5046007229707525
0.5045992115637319
0.5046004206098843


Export the Validation/Developemt set 

In [75]:
# Export the dataset in pickle format
path_sample = os.path.join(projpath, r"02 Data\Prepared Data")
dev = os.path.join(path_sample,r'bcs_data_dev.pkl')
df_train.to_pickle(dev)
val = os.path.join(path_sample,r'bcs_data_val.pkl')
df_test.to_pickle(val)